## 1. INSTALL LIBRARIES AND PACKAGES

In [113]:

# Import the necessary libraries
!pip install datasets
!pip install transformers
!pip install torch
!pip install evaluate
!pip install jiwer
!pip install transformers[torch]
!pip install numpy


In [114]:
##Imports required
import numpy as np
from datasets import load_dataset, Audio, DatasetDict
import torch
import evaluate
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union
from transformers import Seq2SeqTrainingArguments,Seq2SeqTrainer
#from transformers import TrainingArguments, Trainer
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


## 2. IMPORT DATA

In [115]:
dataset = DatasetDict()

# Load the PolyAI dataset.

dataset = load_dataset("PolyAI/minds14", name="en-US", split="train[:80]")

# Remove unnecessary columns


# Split the datasedataset = dataset.remove_columns(['path','english_transcription','intent_class','lang_id'])
t into train and test
dataset = dataset.train_test_split(test_size = 0.2, shuffle=False)


In [116]:
#dataset['train']['audio'][0]

In [117]:
## Resample the dataset to 16 Khz as MCTCT model is trained on 16khz
dataset['train'] = dataset['train'].cast_column("audio", Audio(sampling_rate=16000))
dataset['test'] = dataset['test'].cast_column("audio", Audio(sampling_rate=16000))
# #dataset['train'] = dataset['train'].rename_column("audio", Audio(sampling_rate=16000))
# dataset['train'] = dataset['train'].rename_column("transcription","sentence")
# dataset['test'] = dataset['test'].rename_column("transcription","sentence")

In [118]:
dataset

DatasetDict({
    train: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 64
    })
    test: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 16
    })
})

## 3. DATA PREPROCESSING

In [163]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
processor = WhisperProcessor.from_pretrained("openai/whisper-tiny.en",task="transcribe", model_max_length=225)
model = WhisperForConditionalGeneration.from_pretrained('openai/whisper-tiny.en')
model.to(device)


In [120]:
# Preparing a function to process the entire dataset
# We need to crate two variables with name 'input_featrues'(input array of sound wave in raw foram) and 'labels'(transcription)

def prepare_dataset(batch):
    audio = batch["audio"]

   # batch["input_ids"] = processor.feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"],return_tensor = "pt").input_features[0]
    batch["input_features"] = processor.feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"],return_tensor = "pt").input_features[0]

    #processor.feature_extractor.pad(input_features, return_tensors="pt")
  #  with processor.as_target_processor():
    batch["labels"] = processor.tokenizer(batch["sentence"]).input_ids
    return batch

In [137]:
np.object = object    
encoded_dataset = dataset.map(prepare_dataset, remove_columns=data.column_names["train"],num_proc=4)

#0:   0%|          | 0/16 [00:00<?, ?ex/s]

#2:   0%|          | 0/16 [00:00<?, ?ex/s]

#1:   0%|          | 0/16 [00:00<?, ?ex/s]

#3:   0%|          | 0/16 [00:00<?, ?ex/s]

#1:   0%|          | 0/4 [00:00<?, ?ex/s]

#0:   0%|          | 0/4 [00:00<?, ?ex/s]

#2:   0%|          | 0/4 [00:00<?, ?ex/s]

#3:   0%|          | 0/4 [00:00<?, ?ex/s]

In [138]:
encoded_dataset

DatasetDict({
    train: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 64
    })
    test: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 16
    })
})

In [201]:
# Creating a DataCollatorClass

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union


@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: processor
    padding: Union[bool, str] = "longest"

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
#     def __call__(self, features):# List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need
        # different padding methods
       # print(features)
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")
        
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels
       # print(batch)
        return batch #batch


In [202]:

data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)


In [227]:

# Evalution metric- We will be evaluating our model on word error rate

wer = evaluate.load("wer")

def compute_metrics(pred):
    wer = evaluate.load("wer")
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

# def compute_metrics(pred):
#     wer = evaluate.load("wer")
#     pred_ids = pred.predictions
#     label_ids = pred.label_ids

#     # replace -100 with the pad_token_id
#     label_ids[label_ids == -100] = processor.tokenizer.pad_token_id

#     # we do not want to group tokens when computing the metrics
#     pred_str = processor.tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
#     label_str = processor.tokenizer.batch_decode(label_ids, skip_special_tokens=True)

#     wer = 100 * wer.compute(predictions=pred_str, references=label_str)

#     return {"wer": wer}


#### 4. Training the model

In [228]:
# from transformers import WhisperForConditionalGeneration

# model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")
# model.config.forced_decoder_ids = None
# model.config.suppress_tokens = []
# defining training arguments and trainer
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []
training_args = Seq2SeqTrainingArguments(
    output_dir="seqtoseq-trained",
    gradient_checkpointing=True,
    per_device_train_batch_size=2,
    learning_rate=1e-5,
    warmup_steps=2,
    max_steps=2000,
    fp16=True ,#False ,#True,
    optim='adafactor',
   # group_by_length=True,
    predict_with_generate=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=2,
    eval_steps=100,
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    report_to = ["tensorboard"],
    #data_parallel=False
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["test"],
    tokenizer=processor,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    #data_parallel=False
  #  sampler = None
)


In [229]:
# for feature in encoded_dataset["train"]:
#     print (len(feature['input_ids']))
#     print (len(feature['labels']))
    
#     break

In [ ]:
# Requires GPU for training
trainer.train()

Step,Training Loss,Validation Loss,Wer
100,No log,0.449694,1.000000
200,No log,0.499984,1.000000
300,No log,0.500583,1.000000
400,No log,0.520113,1.000000
500,0.000000,0.547275,1.000000
600,0.000000,0.576372,1.000000


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


In [ ]:
## getting test data
i2 = processor(dataset['test'][6]["audio"]["array"], sampling_rate=16000, return_tensors="pt")
print(f"The input test audio is: {dataset['test'][8]['transcription']}")

# prediction for test data
with torch.no_grad():
    logits = model(**i2.to(device)).logits

predicted_ids = torch.argmax(logits, dim=-1)
transcription = processor.batch_decode(predicted_ids)
print(f'The output prediction is : {transcription[0]}')
     